# Readme embedding evaluation (41 repostitories / test set)

## 1. Loading repository information

In [ ]:
# Downloading repo_info_test.pkl
!pip install gdown
!gdown 1EE5y_u2_RTWk0k1Z2RwELj1eHoc7WeH_

In [ ]:
import pickle

with open("repo_info_test.pkl", "rb") as f:
    repo_info = pickle.load(f)
    f.close()

## 2. Download fine-tuning model

In [ ]:
!pip3 install torch
!pip3 install transformers
# Downloading UniXcoder
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
# !curl -O https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

In [ ]:
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

In [ ]:
unixcoder_model_names = [
    "Lazyhope/unixcoder-nine-advtest",
]

In [ ]:
from unixcoder import UniXcoder

unixcoder_models = {}

# Initialise unixcoder model
for u_name in unixcoder_model_names:
    unixcoder_models[u_name] = UniXcoder(u_name)
    unixcoder_models[u_name].to(device)

## 3. Generating readme embeddings

In [ ]:
# Getting readme embeddings
def get_unixcoder_embeddings(data, model):
    tokens_ids = model.tokenize([data], max_length=512, mode="<encoder-only>")
    source_ids = torch.tensor(tokens_ids).to(device)
    _, embeddings = model(source_ids)
    return embeddings

# Getting other embbeddings
def get_other_embeddings(data_list):
    embeddings = {}

    # Unixcoder model
    for u_name, u_model in tqdm(unixcoder_models.items()):
        print(f" - Using {u_name} model - ")
        embeddings_list = torch.concat([get_unixcoder_embeddings(data, u_model) for data in data_list])
        embeddings[u_name] = embeddings_list

    return embeddings


# Getting embeddings
def get_embeddings(input_data=None, input_mode=None):
    if input_data is None or not input_data:
        return torch.zeros((768,), device=device)

    with torch.no_grad():
        if input_mode == "readme":
            return get_other_embeddings(input_data)


In [ ]:
from tqdm import tqdm

# Generating readme embeddings
for repo_name, repo_dict in tqdm(repo_info.items()):
    print(f" - Generating readme embeddings for {repo_name} - ")
    repo_dict["readme_embeddings"] = get_embeddings(input_data=repo_dict["readme"], input_mode="readme")

## 4. Saving readme embeddings

In [ ]:
save_device = "cpu"
# Change the tensor format to CPU
for repo_name, repo_dict in repo_info.items():
    if type(repo_dict["readme_embeddings"]) is torch.Tensor:
        repo_info[repo_name]["readme_embeddings"] = repo_info[repo_name]["readme_embeddings"].to(save_device)
    else:
        for model_name, model_tensor in repo_dict["readme_embeddings"].items():
            repo_info[repo_name]["readme_embeddings"][model_name] = model_tensor.to(save_device)

In [ ]:
# Check the tensor format
for repo_name, repo_dict in repo_info.items():
    if type(repo_dict["readme_embeddings"]) is torch.Tensor:
        print(repo_dict["readme_embeddings"].get_device())
    else:
        for model_name, model_tensor in repo_dict["readme_embeddings"].items():
             print(repo_info[repo_name]["readme_embeddings"][model_name].get_device())

In [ ]:
# Saving to repo_info_test_readme_embeddings.pkl
with open("repo_info_test_readme_embeddings.pkl", "wb") as f:
    pickle.dump(repo_info, f)
    f.close()

## 5. Calculating similarity

In [ ]:
from torch.nn import CosineSimilarity
from itertools import combinations
import pandas as pd

cossim = CosineSimilarity(dim=0, eps=1e-8)
res = []
num_of_repos = len(repo_info)
num_of_rows = num_of_repos * (num_of_repos - 1) // 2

models = unixcoder_model_names

for repo1, repo2 in tqdm(combinations(repo_info.keys(), 2), total=num_of_rows):
    row = [repo1, repo2, repo_info[repo1]["topic"], repo_info[repo2]["topic"]]
    for model_name in models:

        if type(repo_info[repo1]["readme_embeddings"]) is torch.Tensor:
            embedding1 = repo_info[repo1]["readme_embeddings"]
        else:
            # mean embedding
            embedding1 = torch.mean(repo_info[repo1]["readme_embeddings"][model_name], dim=0)

        if type(repo_info[repo2]["readme_embeddings"]) is torch.Tensor:
            embedding2 = repo_info[repo2]["readme_embeddings"]
        else:
            # mean embedding
            embedding2 = torch.mean(repo_info[repo2]["readme_embeddings"][model_name], dim=0)

        similarity = cossim(embedding1, embedding2).cpu().detach().numpy().item()
        row.append(similarity)

    res.append(row)

df = pd.DataFrame(res, columns=["repo1", "repo2", "topic1", "topic2"] + models)
df

In [ ]:
df.dropna(inplace=True, subset=models)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# Saving similarity calculation
df.to_csv("readme_embedding_evaluation_test_41.csv")

## 6. Evaluating different models

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

model_auc = {}
y_true = df['topic1'] == df['topic2']
fig = plt.figure(figsize=(12, 10))


def roc(model_name):
    y_score = df[model_name]
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    model_auc[model_name] = roc_auc

    plt.plot(fpr, tpr, label=model_name)


for model in models:
    roc(model)

# Order labels by each model's auc score
handles, labels = plt.gca().get_legend_handles_labels()
order = sorted(range(len(labels)), key=lambda k: -model_auc[labels[k]])
ordered_handles = [handles[idx] for idx in order]
ordered_labels = [f'{labels[idx]} (AUC = {model_auc[labels[idx]]:.3f})' for idx in order]
plt.legend(ordered_handles, ordered_labels, loc="lower right")

plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Receiver Operating Characteristic of different models')
# Saving evaluation result
plt.savefig('roc_readme_evaluation_test_41.png')
plt.show()

In [ ]:
model_auc